# Performance Blow-out

## Description
When agents maximize their time when working on these activities by stretching their work, acting as if they are doing the work, or by falsely recording end times.

In [474]:
import pandas as pd
import numpy as np
import altair as alt

In [475]:
log_name = 'propr_test'

fn_logs = {
    'bpic15': '../data/preproc/bpic15.csv',
    'bpic15_ref': '../data/preproc/bpic15_train-70.csv',
    'bpic15_test': '../data/preproc/bpic15_test-30.csv',

    'bpic17': '../data/preproc/bpic2017.csv',

    'propr': '../data/preproc/proprietary.csv',
    'propr_ref': '../data/preproc/proprietary_train-70.csv',
    'propr_test': '../data/preproc/proprietary_test-30.csv'
}

In [476]:
log = pd.read_csv(fn_logs[log_name])
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,_start timestamp,_complete timestamp
0,Precheck,0051574BDF30D681C1257888004DC40D,6/05/2011 16:09,6/05/2011 16:27,090-10-02,Friday,0.0,1.0,0.0,"0,87 € per minute",2011-05-06 16:09:00,2011-05-06 16:27:00
1,Application_Processing_Branches,0051574BDF30D681C1257888004DC40D,9/05/2011 11:14,9/05/2011 11:16,010-23-13,Monday,0.0,1.0,0.0,"1,02 € per minute",2011-05-09 11:14:00,2011-05-09 11:16:00
2,Precheck,0051574BDF30D681C1257888004DC40D,9/05/2011 15:44,9/05/2011 15:58,090-10-02,Monday,0.0,1.0,0.0,"0,87 € per minute",2011-05-09 15:44:00,2011-05-09 15:58:00
3,Processing_Incomplete_Orders,0051574BDF30D681C1257888004DC40D,9/05/2011 16:10,9/05/2011 16:10,010-23-13,Monday,0.0,1.0,0.0,"1,02 € per minute",2011-05-09 16:10:00,2011-05-09 16:10:00
4,Application_Processing_Branches,0051574BDF30D681C1257888004DC40D,10/05/2011 8:52,10/05/2011 9:32,010-23-13,Tuesday,0.0,1.0,0.0,"1,02 € per minute",2011-05-10 08:52:00,2011-05-10 09:32:00
...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,FE3E1E0D3928202EC1257896004B8F00,20/05/2011 17:24,20/05/2011 17:26,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute",2011-05-20 17:24:00,2011-05-20 17:26:00
5286,Check_of_Processing_Applications,FE3E1E0D3928202EC1257896004B8F00,23/05/2011 15:53,23/05/2011 16:06,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",2011-05-23 15:53:00,2011-05-23 16:06:00
5287,Processing_of_Applications,FE3E1E0D3928202EC1257896004B8F00,26/05/2011 9:25,26/05/2011 9:25,010-23-07,Thursday,0.0,2.0,0.0,"1,02 € per minute",2011-05-26 09:25:00,2011-05-26 09:25:00
5288,Archieving,FE3E1E0D3928202EC1257896004B8F00,26/05/2011 10:38,26/05/2011 10:47,010-23-07,Thursday,0.0,2.0,0.0,"1,02 € per minute",2011-05-26 10:38:00,2011-05-26 10:47:00


In [477]:
# add activity duration
log['complete timestamp'] = pd.to_datetime(log['complete timestamp'], dayfirst=True)
log['start timestamp'] = pd.to_datetime(log['start timestamp'], dayfirst=True)
log['activity_duration_minute'] = (
    log['complete timestamp'] -
    log['start timestamp']
).dt.total_seconds() / 60
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,_start timestamp,_complete timestamp,activity_duration_minute
0,Precheck,0051574BDF30D681C1257888004DC40D,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,0.0,1.0,0.0,"0,87 € per minute",2011-05-06 16:09:00,2011-05-06 16:27:00,18.0
1,Application_Processing_Branches,0051574BDF30D681C1257888004DC40D,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,0.0,1.0,0.0,"1,02 € per minute",2011-05-09 11:14:00,2011-05-09 11:16:00,2.0
2,Precheck,0051574BDF30D681C1257888004DC40D,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,0.0,1.0,0.0,"0,87 € per minute",2011-05-09 15:44:00,2011-05-09 15:58:00,14.0
3,Processing_Incomplete_Orders,0051574BDF30D681C1257888004DC40D,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,0.0,1.0,0.0,"1,02 € per minute",2011-05-09 16:10:00,2011-05-09 16:10:00,0.0
4,Application_Processing_Branches,0051574BDF30D681C1257888004DC40D,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,0.0,1.0,0.0,"1,02 € per minute",2011-05-10 08:52:00,2011-05-10 09:32:00,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,FE3E1E0D3928202EC1257896004B8F00,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute",2011-05-20 17:24:00,2011-05-20 17:26:00,2.0
5286,Check_of_Processing_Applications,FE3E1E0D3928202EC1257896004B8F00,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",2011-05-23 15:53:00,2011-05-23 16:06:00,13.0
5287,Processing_of_Applications,FE3E1E0D3928202EC1257896004B8F00,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,0.0,2.0,0.0,"1,02 € per minute",2011-05-26 09:25:00,2011-05-26 09:25:00,0.0
5288,Archieving,FE3E1E0D3928202EC1257896004B8F00,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,0.0,2.0,0.0,"1,02 € per minute",2011-05-26 10:38:00,2011-05-26 10:47:00,9.0


In [478]:
# Does "Cost" differ by activity or resource?
# NOTE: it seems that all resources are associated with a stable cost, except
# resource 004-9-1, which has two costs
# df = log[['resource', 'Cost']].groupby('resource').agg(cost_options=pd.NamedAgg('Cost', aggfunc='nunique'))
# df[df['cost_options'] > 1]


## Performance-PB-1

### Profile work by activity types and/or case types

In [479]:
# proprietary: 
# case types as types of loan
log['case_type'] = log['Type of loan; 0 = normal; 1 = special'].apply(
    lambda x: 'special loan' if x == 1 else 'normal loan'
)

case_types = list(log['case_type'].unique())
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,_start timestamp,_complete timestamp,activity_duration_minute,case_type
0,Precheck,0051574BDF30D681C1257888004DC40D,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,0.0,1.0,0.0,"0,87 € per minute",2011-05-06 16:09:00,2011-05-06 16:27:00,18.0,normal loan
1,Application_Processing_Branches,0051574BDF30D681C1257888004DC40D,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,0.0,1.0,0.0,"1,02 € per minute",2011-05-09 11:14:00,2011-05-09 11:16:00,2.0,normal loan
2,Precheck,0051574BDF30D681C1257888004DC40D,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,0.0,1.0,0.0,"0,87 € per minute",2011-05-09 15:44:00,2011-05-09 15:58:00,14.0,normal loan
3,Processing_Incomplete_Orders,0051574BDF30D681C1257888004DC40D,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,0.0,1.0,0.0,"1,02 € per minute",2011-05-09 16:10:00,2011-05-09 16:10:00,0.0,normal loan
4,Application_Processing_Branches,0051574BDF30D681C1257888004DC40D,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,0.0,1.0,0.0,"1,02 € per minute",2011-05-10 08:52:00,2011-05-10 09:32:00,40.0,normal loan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,FE3E1E0D3928202EC1257896004B8F00,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute",2011-05-20 17:24:00,2011-05-20 17:26:00,2.0,normal loan
5286,Check_of_Processing_Applications,FE3E1E0D3928202EC1257896004B8F00,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",2011-05-23 15:53:00,2011-05-23 16:06:00,13.0,normal loan
5287,Processing_of_Applications,FE3E1E0D3928202EC1257896004B8F00,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,0.0,2.0,0.0,"1,02 € per minute",2011-05-26 09:25:00,2011-05-26 09:25:00,0.0,normal loan
5288,Archieving,FE3E1E0D3928202EC1257896004B8F00,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,0.0,2.0,0.0,"1,02 € per minute",2011-05-26 10:38:00,2011-05-26 10:47:00,9.0,normal loan


### Determine groups of comparable resources

In [480]:
# proprietary
log['department'] = log['resource'].apply(
    lambda x: x.split('-')[0]
)
log['department_role'] = log['resource'].apply(
    lambda x: x.split('-')[0] + '-' + x.split('-')[1]
)
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,_start timestamp,_complete timestamp,activity_duration_minute,case_type,department,department_role
0,Precheck,0051574BDF30D681C1257888004DC40D,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,0.0,1.0,0.0,"0,87 € per minute",2011-05-06 16:09:00,2011-05-06 16:27:00,18.0,normal loan,090,090-10
1,Application_Processing_Branches,0051574BDF30D681C1257888004DC40D,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,0.0,1.0,0.0,"1,02 € per minute",2011-05-09 11:14:00,2011-05-09 11:16:00,2.0,normal loan,010,010-23
2,Precheck,0051574BDF30D681C1257888004DC40D,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,0.0,1.0,0.0,"0,87 € per minute",2011-05-09 15:44:00,2011-05-09 15:58:00,14.0,normal loan,090,090-10
3,Processing_Incomplete_Orders,0051574BDF30D681C1257888004DC40D,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,0.0,1.0,0.0,"1,02 € per minute",2011-05-09 16:10:00,2011-05-09 16:10:00,0.0,normal loan,010,010-23
4,Application_Processing_Branches,0051574BDF30D681C1257888004DC40D,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,0.0,1.0,0.0,"1,02 € per minute",2011-05-10 08:52:00,2011-05-10 09:32:00,40.0,normal loan,010,010-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,FE3E1E0D3928202EC1257896004B8F00,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute",2011-05-20 17:24:00,2011-05-20 17:26:00,2.0,normal loan,000,000-3
5286,Check_of_Processing_Applications,FE3E1E0D3928202EC1257896004B8F00,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",2011-05-23 15:53:00,2011-05-23 16:06:00,13.0,normal loan,000,000-2
5287,Processing_of_Applications,FE3E1E0D3928202EC1257896004B8F00,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,0.0,2.0,0.0,"1,02 € per minute",2011-05-26 09:25:00,2011-05-26 09:25:00,0.0,normal loan,010,010-23
5288,Archieving,FE3E1E0D3928202EC1257896004B8F00,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,0.0,2.0,0.0,"1,02 € per minute",2011-05-26 10:38:00,2011-05-26 10:47:00,9.0,normal loan,010,010-23


In [481]:
# proprietary
# select resources from the same department_role
sel_resources = set(log.loc[
    log['department_role'] == '010-24',
    'resource'
].unique())

### Profile group member performance by activity duration

In [482]:
# use average activity duration for the same activity
res_performance = log.groupby(
    ['resource', 'activity', 'case_type']
).agg(
    mean_activity_duration_minute=pd.NamedAgg(
        'activity_duration_minute', 'mean'
    )
).reset_index()
res_performance = res_performance[
    res_performance['resource'].isin(sel_resources)
]
res_performance

,resource,activity,case_type,mean_activity_duration_minute
202,010-24-01,Archieving,normal loan,13.372093
203,010-24-01,Archieving,special loan,12.666667
204,010-24-01,Check_of_Processing_Applications,normal loan,0.500000
205,010-24-01,Check_of_Processing_Applications,special loan,0.000000
206,010-24-01,End,normal loan,0.000000
...,...,...,...,...
308,010-24-26,Processing_of_Applications_Sonderkredit,special loan,2.833333
309,010-24-26,Revision,special loan,1.000000
310,010-24-26,Rework_Branches,normal loan,9.350000
311,010-24-26,Rework_Branches,special loan,18.000000


### Identify resources who spent significantly longer time on the same type of work compared to others

In [483]:
data = res_performance[
    (res_performance['resource'].isin(sel_resources)) &
    (res_performance['case_type'] == 'special loan')
]
alt.Chart(data).mark_rect().encode(
    y=alt.Y('resource:N').title('Resource'),
    x=alt.X('activity:N').title('Activity'),
    color=alt.Color('mean_activity_duration_minute').title(
        'Average activity duration (minutes)'
    ),
    tooltip=['resource', 'activity', 'case_type', 'mean_activity_duration_minute']
)

alt.Chart(...)

In [484]:
log_data = log[
    (log['resource'].isin(sel_resources)) &
    (log['case_type'] == 'special loan')
]

res_data = res_performance[
    (res_performance['resource'].isin(sel_resources)) &
    (res_performance['case_type'] == 'special loan')
]

charts = []
for activity in res_data['activity'].unique():
    # Show only visualizations of certain activities
    if activity not in {
        'Administrative_Tasks',
        'Check_of_Processing_Applications',
        'Preparation_Special_Processing',
        'Processing_of_Applications_Sonderkredit',
        'Archieving',
        'End',
        'Precheck',
        'Processing_of_Applications'
    }:
        continue
    threshold = log_data['activity_duration_minute'].quantile(0.75) + 1.5 * (
        log_data['activity_duration_minute'].quantile(0.75) -
        log_data['activity_duration_minute'].quantile(0.25)
    )
    print('Activity "{}" mean duration time threshold (Q3 + 1.5IQR): {}'.format(
        activity, threshold
    ))
    subset = res_data[res_data['activity'] == activity]
    selection = alt.selection_point(fields=['resource'], bind='legend')
    charts.append(
        alt.layer(
            alt.Chart(log_data[log_data['activity'] == activity]).mark_boxplot(
                outliers={'size': 0}
            ).encode(
                y='activity',
                x=alt.X('activity_duration_minute').title(
                    'Activity duration time (minute)'
                ),
                color=alt.value('grey'),
                opacity=alt.value(0.5)
            ),
            alt.Chart(log_data[log_data['activity'] == activity]).mark_circle().encode(
                y='activity',
                x='activity_duration_minute',
                color='resource',
                tooltip=['resource', 'activity', 'activity_duration_minute'],
                opacity=alt.condition(selection, alt.value(1), alt.value(0))
            ).add_params(
                selection
            )
        )
    )
alt.vconcat(*charts)

Activity "Archieving" mean duration time threshold (Q3 + 1.5IQR): 45.0
Activity "Check_of_Processing_Applications" mean duration time threshold (Q3 + 1.5IQR): 45.0
Activity "End" mean duration time threshold (Q3 + 1.5IQR): 45.0
Activity "Precheck" mean duration time threshold (Q3 + 1.5IQR): 45.0
Activity "Processing_of_Applications" mean duration time threshold (Q3 + 1.5IQR): 45.0
Activity "Administrative_Tasks" mean duration time threshold (Q3 + 1.5IQR): 45.0
Activity "Preparation_Special_Processing" mean duration time threshold (Q3 + 1.5IQR): 45.0
Activity "Processing_of_Applications_Sonderkredit" mean duration time threshold (Q3 + 1.5IQR): 45.0


alt.VConcatChart(...)